In [1]:
from counterfactuals.explainers import Cadex
from counterfactuals.visualization import compare
from data import GermanData
from tensorflow import keras

ModuleNotFoundError: No module named 'counterfactuals.models'

In [ ]:
german_data = GermanData('data/input_german.csv', 'data/labels_german.csv')
model = keras.models.load_model('models/model_german')
test_X = german_data.X_test.iloc[6]
cadex = Cadex(model)
cf = cadex.generate(test_X)

In [ ]:
df = compare(test_X, cf.iloc[0])
df

,X,X',change
age,-1.008682,0.421559,1.430241
account_status_no checking account,-0.818625,0.627561,1.446185
credit_history_critical account,-0.664411,0.812965,1.477375
purpose_car (used),-0.328684,1.271210,1.599894
account_status_>= 200 DM,-0.258199,1.172036,1.430235


In [ ]:
test_X = german_data.input.iloc[6]
cadex = Cadex(model, transform=german_data.scale, inverse_transform=german_data.unscale)
cf = cadex.generate(test_X)
compare(test_X, cf.iloc[0])

,X,X',change
account_status_< 0 DM,0,9.823950e-01,9.823950e-01
account_status_>= 200 DM,0,6.938894e-18,6.938894e-18
credit_history_no credits taken,0,4.134390e-01,4.134390e-01
purpose_business,0,1.387779e-17,1.387779e-17
purpose_car (new),0,8.729189e-01,8.729189e-01
housing_for free,0,1.387779e-17,1.387779e-17
job_unskilled - resident,0,8.093202e-01,8.093202e-01
duration,24,4.907340e+01,2.507340e+01


In [ ]:
test_X = german_data.X_test.iloc[10]
cadex = Cadex(model, n_changed=8)
cf = cadex.generate(test_X)
compare(test_X, cf.iloc[0])

,X,X',change
duration,1.232869,2.216195,0.983326
credit,2.348373,3.328096,0.979724
installment_percent,-1.799472,-0.804376,0.995096
savings_< 100 DM,-1.237597,-0.232609,1.004987
employment_< 1 year,-0.466536,0.503254,0.969790
job_management/self-employed,-0.413888,0.594769,1.008657
other_installment_plans_bank,-0.403473,0.599126,1.002600
credit_history_no credits taken,-0.193942,0.788318,0.982260


In [ ]:
from counterfactuals.explainers import ECE

ece = ECE(5, columns=list(german_data.X_train.columns), bces=[Cadex(model, n_changed=i) for i in range(10, 20)], dist=2, h=8, lambda_=0.2, n_jobs=1)

ece.generate(german_data.X_test.iloc[0])


,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,-0.176657,-0.543414,0.070758,-0.781879,-0.322868,0.985887,-0.344173,-0.534718,-0.536582,-0.258199,...,-0.51558,0.879477,-0.810133,-0.197386,0.266832,1.437057,-0.454573,-0.466536,-0.588898,-0.266365
1,-0.176450,-0.543210,0.070966,-0.781879,-0.322868,0.985887,-0.343962,-0.534511,-0.536373,-0.258199,...,-0.51558,0.879681,-0.810133,-0.197386,0.267038,1.437057,-0.454573,-0.466536,-0.588898,-0.266365


In [ ]:
ece.get_aggregated_cfs()

,duration,credit,installment_percent,residence_duration,age,existing_credits,people_maintained,account_status_0..200 DM,account_status_< 0 DM,account_status_>= 200 DM,...,job_unskilled - resident,phone_none,"phone_yes, registered under the customers name",foreign_no,foreign_yes,employment_1..4 years,employment_4..7 years,employment_< 1 year,employment_>= 7 years,employment_unemployed
0,-0.176657,-0.543414,0.070758,-0.781879,-0.322868,0.985887,-0.344173,-0.534718,-0.536582,-0.258199,...,-0.515580,0.879477,-0.810133,-0.197386,0.266832,1.437057,-0.454573,-0.466536,-0.588898,-0.266365
1,-0.176450,-0.543210,0.070966,-0.781879,-0.322868,0.985887,-0.343962,-0.534511,-0.536373,-0.258199,...,-0.515580,0.879681,-0.810133,-0.197386,0.267038,1.437057,-0.454573,-0.466536,-0.588898,-0.266365
2,-0.176243,-0.543007,0.071174,-0.781879,-0.322868,0.985887,-0.343753,-0.534304,-0.536165,-0.258199,...,-0.515580,0.879885,-0.810133,-0.197386,0.267243,1.437057,-0.454573,-0.466536,-0.588898,-0.196148
3,-0.176037,-0.542803,0.071382,-0.781879,-0.322868,0.985887,-0.343544,-0.534097,-0.535957,-0.258199,...,-0.515580,0.880089,-0.810133,-0.197386,0.267448,1.437057,-0.454573,-0.466536,-0.588898,-0.195937
4,-0.175829,-0.542597,0.071590,-0.781879,-0.322868,0.985887,-0.343334,-0.533888,-0.535748,-0.258199,...,-0.515580,0.880295,-0.810133,-0.197386,0.267655,1.437057,-0.454573,-0.466536,-0.588898,-0.195726
5,-0.175566,-0.542094,0.071763,-0.781879,-0.322868,0.985887,-0.343250,-0.533670,-0.535597,-0.258199,...,-0.515580,0.880770,-0.810133,-0.197386,0.268023,1.437057,-0.454573,-0.466536,-0.588898,-0.195728
6,-0.175359,-0.541887,0.071970,-0.781879,-0.322868,0.985887,-0.343043,-0.533463,-0.535390,-0.258199,...,-0.515580,0.880977,-0.810133,-0.197386,0.268230,1.437057,-0.454573,-0.466536,-0.588898,-0.195521
7,-0.175211,-0.541985,0.072211,-0.781879,-0.322868,0.985887,-0.342710,-0.533269,-0.535126,-0.258199,...,-0.515580,0.880908,-0.740336,-0.197386,0.268270,1.437057,-0.454573,-0.466536,-0.588898,-0.195098
8,-0.175002,-0.541777,0.072421,-0.781879,-0.322868,0.985887,-0.342500,-0.533060,-0.534917,-0.258199,...,-0.515580,0.881116,-0.740131,-0.197386,0.268478,1.437057,-0.454573,-0.466536,-0.588898,-0.194888
9,-0.174795,-0.541571,0.072628,-0.781879,-0.322868,0.985887,-0.342292,-0.532853,-0.534709,-0.258199,...,-0.515580,0.881322,-0.739929,-0.197386,0.268685,1.437057,-0.454573,-0.466536,-0.588898,-0.194679


In [ ]:
german_data.X_train.iloc[0]

duration                 0.246820
credit                  -0.431730
installment_percent     -1.799472
residence_duration      -1.692363
age                     -1.180136
                           ...   
employment_1..4 years   -0.695866
employment_4..7 years   -0.454573
employment_< 1 year      2.143456
employment_>= 7 years   -0.588898
employment_unemployed   -0.266365
Name: 405, Length: 61, dtype: float64